## **Boosting Marketing Efficiency: Targeted Bank Campaign by Customer Subscription Behavior**

**Overall Project Objective:** 

Develop a data-driven marketing strategy that maximizes return on investment (ROI) by identifying optimal trade-off between broad customer outreach and precision marketing using predictive modeling.

**Notebook 3 of 3: Feature Importance & Business Impact Analysis

This notebook dives deeper into our final model and derive business insight that fulfills our overall project objective.

### **Setup & Data Load**

Libraries, the final model, and the test data load.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.metrics import precision_recall_curve, roc_curve


In [ ]:
# load the trained model (random forest)
file_path = '../models/tuned_rf_mod.joblib'
tuned_rf_mod = joblib.load(file_path)

**-Data Overview-**

Only the test data files ('X_test_scaled', 'y_test') will be used in this notebook. 

In [3]:
# load the test data
X_test = pd.read_csv('../data/X_test_scaled.csv')
y_test = pd.read_csv('../data/y_test.csv').squeeze()

print("Test set shape: ", X_test.shape)

Test set shape:  (905, 37)


### **Feature Importance Analysis**

### **Precision-Recall Trade-Off Analysis**

### **Conclusion & Recommendation**